In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Weather Analysis results into DataFrame
* Load the csv exported from Weather Analysis to a DataFrame

In [2]:
# Display the DataFrame by calling the stored csv 
city_data_df = pd.read_csv("../Weather Analysis/output_data/cities.csv", encoding="ISO-8859-1")
city_data_df

# New dataframe with columns we want
formatted_df = city_data_df[["name", "coord.lat","coord.lon","main.temp_max","main.humidity","clouds.all","wind.speed","sys.country","dt"]]

#Renaming columns
renamed_df = formatted_df.rename(columns={"name":"City","coord.lat":"Lat","coord.lon":"Lng","main.temp_max":"Max Temp","main.humidity":"Humidity","clouds.all":"Cloudiness","wind.speed":"Wind Speed","sys.country":"Country","dt":"Date"})

#take only rows where City column is not NaN value.
renamed_df = renamed_df[renamed_df['City'].notna()]
renamed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,73.80,86.0,1.0,6.91,US,1.627750e+09
1,Longyearbyen,78.2186,15.6401,44.44,70.0,75.0,12.66,SJ,1.627750e+09
2,Ekibastuz,51.6667,75.3667,73.45,36.0,26.0,12.19,KZ,1.627750e+09
3,Bredasdorp,-34.5322,20.0403,56.52,82.0,94.0,3.44,ZA,1.627750e+09
4,Punta Arenas,-53.1500,-70.9167,41.11,60.0,0.0,13.80,CL,1.627750e+09
...,...,...,...,...,...,...,...,...,...
591,Tadine,-21.5500,167.8833,66.70,57.0,6.0,18.28,NC,1.627750e+09
593,Kargil,34.5667,76.1000,63.12,51.0,81.0,8.01,IN,1.627750e+09
594,Vao,-22.6667,167.4833,67.15,57.0,0.0,24.18,NC,1.627750e+09
595,Puerto del Rosario,28.5004,-13.8627,78.48,73.0,20.0,21.85,ES,1.627750e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Create initial figure
fig = gmaps.figure()

# Create humidity Heatmap layer
locations = renamed_df[["Lat", "Lng"]]
humidity_heat_layer = gmaps.heatmap_layer(locations, weights=renamed_df["Humidity"])
fig.add_layer(humidity_heat_layer)

#Visualize
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
data_df = renamed_df.dropna()
data_df.count()
condition_1 = (data_df["Max Temp"] < 80) & (data_df["Max Temp"] > 70)
condition_2 = (data_df["Wind Speed"] < 10)
condition_3 = (data_df["Cloudiness"] == 0)
ideal_weather_df = data_df.loc[condition_1 & condition_2 & condition_3].reset_index(drop=True)
print(ideal_weather_df.describe())
print(ideal_weather_df.head())

             Lat         Lng  Max Temp   Humidity  Cloudiness  Wind Speed  \
count   4.000000    4.000000   4.00000   4.000000         4.0    4.000000   
mean   41.466875  -19.213950  74.90500  58.000000         0.0    5.705000   
std    14.119982   68.022716   3.50385   9.966611         0.0    3.008815   
min    30.483300 -115.950000  71.56000  48.000000         0.0    3.000000   
25%    32.120850  -41.812500  72.50500  50.250000         0.0    3.885000   
50%    36.958750    3.200000  74.31500  58.000000         0.0    4.965000   
75%    46.304775   25.798550  76.71500  65.750000         0.0    6.785000   
max    61.466700   32.694200  79.43000  68.000000         0.0    9.890000   

               Date  
count  4.000000e+00  
mean   1.627750e+09  
std    5.131195e+01  
min    1.627750e+09  
25%    1.627750e+09  
50%    1.627750e+09  
75%    1.627750e+09  
max    1.627750e+09  
           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0  San QuintÃ­n  30.4833 -115.9500    

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    response = requests.get(base_url, params=params)

    # convert to json
    hotel_json = response.json()

     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
         hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
#         cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
#         cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")

In [7]:
hotel_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,San QuintÃ­n,30.4833,-115.9500,75.81,68.0,0.0,9.89,MX,1.627750e+09,Old Mill Hotel
1,Nokia,61.4667,23.5000,71.56,48.0,0.0,3.00,FI,1.627750e+09,Scandic Eden
2,Ponta do Sol,32.6667,-17.1000,79.43,65.0,0.0,5.75,PT,1.627750e+09,Enotel Sunset Bay
3,Safranbolu,41.2508,32.6942,72.82,51.0,0.0,4.18,TR,1.627750e+09,Baglar Saray Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))